# 1. Introduction:

## 1.1 Research Topic

Stereochemistry is an important concept in organic chemistry for understanding molecules' behavior in biological systems. Indeed, the spatial arrangement of atoms in a molecule can significantly influence how that molecule interacts with enzymes and other biomolecules. Stereoisomers—compounds that have the same molecular formula but differ in the 3D orientation of their atoms—often exhibit different properties. A thorough understanding of stereochemistry is therefore essential in fields such as pharmaceutical chemistry, where the difference between isomers can mean the difference between a therapeutic and a toxic effect.

## 1.2 Settings: Niche

While there are tools for drawing molecules and naming stereoisomers, few are designed to be interactive learning tools. Most tools and materials explain stereochemistry in a serious and technical way, but they are not fun or easy to use. Because of that, students often find it hard to really understand the topic. Furthermore, current tools often do not simplify the challenge of identifying stereocenters or visualizing all possible stereoisomers for a given molecule, especially from linear notations like SMILES.

## 1.3 Problem

Students often struggle when it comes to understanding and visualizing stereoisomers, particularly when translating 2D representations into 3D structures. As a result, students may fail to understand how chirality and molecular orientation influence chemical behavior. After struggling with the concepts of stereochemistry, the idea of this project seemed obvious to us: we wanted to create an interface to get acquainted with stereoisomers, nomenclature, and chirality. Thus, in this project, we tried to put as many functionalities as possible into a Streamlit interface, which we hope will serve as a helpful and intuitive tool for users.

## 1.4 Solution

This project presents a Python-based tool that automatically detects chiral centers and calculates the number of possible stereoisomers of a given molecule. Using cheminformatics libraries such as RDKit, the tool processes SMILES input and outputs an intuitive stereochemical analysis. The aim is to create an accessible, educational resource that not only aids students in mastering stereochemistry but also serves as a basis for further computational chemistry applications.